In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from typing import Optional

DATASETS_DIR = "./datasets"
class GPUDataset(Dataset):
    def __init__(self, dataset: Dataset, transform: Optional[transforms.Compose], device: str):
        self.device = device
        self.transform = transform
        shape = transform(dataset[0][0]).shape

        self.data = torch.empty((len(dataset), *transform(dataset[0][0]).shape), device=device)
        self.targets = torch.empty(len(dataset), dtype=torch.long, device=device)

        for i, (img, target) in enumerate(dataset):
            self.data[i] = transform(img).to(device)
            self.targets[i] = torch.tensor(target).to(device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]


def get_dataset(dataset_name: str = "FashionMNIST") -> tuple[Dataset, Dataset]:
    if dataset_name == "FashionMNIST":
        train_dataset = datasets.FashionMNIST(root=DATASETS_DIR, train=True, download=True)
        test_dataset = datasets.FashionMNIST(root=DATASETS_DIR, train=False, download=True)
    elif dataset_name == "CIFAR10":
        train_dataset = datasets.CIFAR10(root=DATASETS_DIR, train=True, download=True)
        test_dataset = datasets.CIFAR10(root=DATASETS_DIR, train=False, download=True)

    return train_dataset, test_dataset


In [2]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, Subset, random_split
import numpy as np


def get_dataloaders(
    dataset_name: str = "FashionMNIST",
    batch_size: int = 32,
    val_split: float = 0.2,
    seed: int = 42,
    num_workers: int = 0,
    device: str = "cuda",
) -> tuple[Dataset, DataLoader, DataLoader, DataLoader]:

    torch.manual_seed(seed)

    if dataset_name == "FashionMNIST":
        train_dataset, test_dataset = get_dataset(dataset_name=dataset_name)
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,)),
            ]
        )
    elif dataset_name == "CIFAR10":
        (0.5, 0.5, 0.5)
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]
        )
        train_dataset, test_dataset = get_dataset(dataset_name=dataset_name)
    else:
        raise ValueError(f"Unknown dataset name: {dataset_name}")

    val_size = int(val_split * len(train_dataset))
    train_size = len(train_dataset) - val_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    train_dataset = GPUDataset(train_dataset, transform, device)
    val_dataset = GPUDataset(val_dataset, transform, device)
    test_dataset = GPUDataset(test_dataset, transform, device)

    train_dataloader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )
    val_dataloader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )
    test_dataloader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )

    return train_dataset, train_dataloader, val_dataloader, test_dataloader


def get_subset_loader(
    dataset: Dataset, mask: np.ndarray, batch_size: int = 32, num_workers: int = 0
) -> DataLoader:
    assert mask.dtype == bool, "Mask must be a boolean array."
    assert len(mask) == len(dataset), "Mask and dataset must have the same length."
    indices = np.where(mask)[0]
    subset = Subset(dataset, indices)
    return DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [12]:
from torch import optim
from tqdm import tqdm
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
import numpy as np
BATCHES = 150



def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    target_iterations: int = 2,
    learning_rate: float = 1e-3,
    device: torch.device = torch.device("cuda"),
) -> None:
    """
    Trains the given neural network model on the provided dataset, balancing
    the number of epochs and iterations to ensure fair training across datasets
    of different sizes.

    Args:
        model (nn.Module): The neural network model to train.
        train_loader (DataLoader): DataLoader providing the training dataset.
        target_iterations (int, optional): Target number of training iterations.
            Default is 1000.
        learning_rate (float, optional): Learning rate for the optimizer. Default is 1e-3.
        device (torch.device, optional): Device for training (default is CUDA if available).

    Returns:
        None
    """
    NUM_BATCHES = BATCHES * target_iterations
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

    model.train()
    model.to(device)

    train_iter = iter(train_loader)

    batch_count = 0
    with tqdm(total=NUM_BATCHES, desc="Training Progress") as pbar:
        while batch_count < NUM_BATCHES:
            try:
                images, labels = next(train_iter)
            except StopIteration:
                train_iter = iter(train_loader)
                images, labels = next(train_iter)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            batch_count += 1
            pbar.update(1)


def loss_fn(balanced_accuracy: float, alpha: float, beta: float, S: int, D: int) -> float:
    """
    Computes the loss function based on Balanced Accuracy and a subset penalty.

    Args:
        balanced_accuracy (float): The Balanced Accuracy of the model on the validation dataset.
        alpha (float): Weight for the Balanced Accuracy component of the loss.
        beta (float): Weight for the subset size penalty component of the loss.
        S (int): Size of the current subset.
        D (int): Size of the entire dataset.

    Returns:
        float: The computed loss value.
    """
    return alpha * balanced_accuracy - beta * (S / D)


def validate_model(
    model: nn.Module,
    val_loader: DataLoader,
    alpha: float = 1.0,
    beta: float = 0.5,
    S: int = 10,
    D: int = 1000,
    device: torch.device = torch.device("cuda"),
    compute_confusion: bool = True,
) -> tuple[float, float, None | np.ndarray]:
    """
    Validates the model on the validation dataset and computes the loss based on Balanced Accuracy.

    Args:
        model (nn.Module): The neural network model to validate.
        val_loader (DataLoader): DataLoader for the validation dataset.
        alpha (float): Weight for Balanced Accuracy in the loss function.
        beta (float): Weight for subset size penalty in the loss function.
        S (int): Size of the current subset.
        D (int): Size of the full dataset.
        device (torch.device): Device for computation ('cuda' or 'cpu').

    Returns:
        Tuple[float, float]: Loss and Balanced Accuracy.
    """
    device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
    model.eval()
    model.to(device)

    all_labels = []
    all_predictions = []

    with torch.inference_mode():
        for images, labels in val_loader:
            output = model(images)
            _, predicted = torch.max(output.data, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    balanced_accuracy = balanced_accuracy_score(all_labels, all_predictions)
    loss = loss_fn(balanced_accuracy, alpha, beta, S, D)
    if not compute_confusion:
        return loss, balanced_accuracy

    confusion = confusion_matrix(all_labels, all_predictions)
    return loss, balanced_accuracy, confusion

In [4]:
from torch import nn
from torch.nn import functional as F


class CNN3Channel(nn.Module):
    def __init__(self):
        super(CNN3Channel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
num_workers = 0
train_dataset, train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
    dataset_name="CIFAR10", num_workers=num_workers, device=device
)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
len(train_dataloader)


1250

In [7]:
len(val_dataloader)

313

# INITIAL INDYVIDUAL SIZE

In [17]:
for iter_number in range (1, 10):
    for percent in range(5, 101, 5):
        score = 0
        for i in range(3):
            model = CNN3Channel()
            train_dataloader = get_subset_loader(
                train_dataset,
                np.random.choice([True, False], size=len(train_dataset), p=[percent / 100, 1 - percent / 100]),)
            train_model(model, train_dataloader, target_iterations=iter_number)
            loss, balanced_accuracy = validate_model(model, test_dataloader, compute_confusion=False)
            score += balanced_accuracy
        score /= 3
        print(f"Percent: {percent}, target_iterations: {iter_number}, Score: {score}")


Training Progress: 100%|██████████| 150/150 [00:00<00:00, 251.12it/s]


Percent: 5, target_iterations: 1, Score: 0.34736666666666666


Training Progress: 100%|██████████| 150/150 [00:00<00:00, 352.57it/s]


Percent: 10, target_iterations: 1, Score: 0.3524


Training Progress: 100%|██████████| 150/150 [00:00<00:00, 348.04it/s]


...

Percent: 90, target_iterations: 9, Score: 0.5734666666666667


Training Progress: 100%|██████████| 1350/1350 [00:03<00:00, 344.46it/s]


Percent: 95, target_iterations: 9, Score: 0.5559666666666666


Training Progress: 100%|██████████| 1350/1350 [00:04<00:00, 332.02it/s]


Percent: 100, target_iterations: 9, Score: 0.5730333333333333
